In [1]:
from helpers import *
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species
from rascal.representations import SphericalExpansion
from rascal.utils import ClebschGordanReal, compute_lambda_soap, spherical_expansion_reshape, xyz_to_spherical

In [2]:
#load data get indices process tensors
#build lsoaps, 0,1,2
##get an optimal radial basis
##do PCA,FPS to reduce number of features?

##test
#
#return optimal radial
#error metric

In [29]:
PATH_TRAIN = "./train_tensor/CSD-3k+S546_shift_tensors.xyz"
PATH_TEST = "./test_tensor/CSD-500+104-7_shift_tensors.xyz"

#
structures_train, tensors_train = load_CSD_data(PATH_TRAIN,prop_string="cs_tensor",random_subsample=100)
structures_test, tensors_test = load_CSD_data(PATH_TEST,prop_string="cs_tensor")

for structure in structures_train: mask_center_atoms_by_species(structure,species_select=[1])
for structure in structures_test: mask_center_atoms_by_species(structure,species_select=[1])

train_tensors = np.array([tensor for structure in structures_train for tensor in structure.arrays["cs_tensor"][structure.arrays["center_atoms_mask"]]])
test_tensors = np.array([tensor for structure in structures_test for tensor in structure.arrays["cs_tensor"][structure.arrays["center_atoms_mask"]]])



In [28]:
tensors_test.shape

(76968, 9)

In [17]:
#these are nearly optimal parameters of the shiftml 2.0

hypers = dict(interaction_cutoff=4.5,
              max_radial=9,
              max_angular=9,
              gaussian_sigma_constant=0.3,
              gaussian_sigma_type="Constant",
              cutoff_function_type="RadialScaling",
              cutoff_smooth_width=0.5,
              cutoff_function_parameters=
                    dict(
                            rate=1,
                            scale=2.0,
                            exponent=3
                        ),
              radial_basis="GTO",
              optimization=
                    dict(
                            Spline=dict(
                               accuracy=1.0e-05
                            )
                        ),
              compute_gradients=False
              )

#i tried scale=3.0 and exponent=6 aswell

#hypers = get_optimal_radial_basis_hypers(hypers, structures_train+structures_test, expanded_max_radial=50)

In [18]:
def build_lambda(hypers,structures_train, CG):
    structures_train = list(filter(None, structures_train))
    spex = SphericalExpansion(**hypers)
    feats = spex.transform(structures_train).get_features(spex)
    ref_feats = 1e6*spherical_expansion_reshape(feats, **hypers)
    lsoap = compute_lambda_soap(ref_feats, CG, 2, 1)
    return lsoap

In [19]:
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [20]:
grouped = grouper(structures_train, 10)

In [21]:
CG = ClebschGordanReal(lmax=hypers["max_angular"]) 

In [22]:
CG.shape

AttributeError: 'ClebschGordanReal' object has no attribute 'shape'

In [23]:
this = [build_lambda(hypers,group,CG) for group in grouped]

In [31]:
import scipy as sp

In [ ]:
lsr = np.moveaxis(this_,-1,1).reshape((this_.shape[0]*this_.shape[-1], -1))
u, s, vt = sp.linalg.svd(lsr, full_matrices=False)

In [25]:
this_ = np.concatenate(this)

In [ ]:
this_.shape

In [26]:
this_.nbytes * 1e-06

5429.5109999999995

In [32]:
lsoap.nbytes * 1e-06

0.0

In [33]:
lsoap.size * lsoap.itemsize

0

In [34]:
lsoap

array([], shape=(20076, 5, 12, 5, 12, 0, 5), dtype=float64)